In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
34,application_1596730828793_0062,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import boto3
import datetime
import pandas as pd
from hops import hdfs

# import os

# from pyspark.sql import SparkSession
# from pyspark.sql import Row

# os.listdir()

In [7]:
1

1

In [8]:
df.take(1)

[Row(timestamp='2001-01-09 20:09:00', FPAC=0, BLAC=0, CTAC=0, TH=0, MH=0, EGT_1=30.5, EGT_2=29.5, EGT_3=31.0, EGT_4=30.5, IVV=0, GS=0, TRK=0, TRKM=0, DA=0, POVT=0, WS=0, MW=0, DFGS=1, WD=0, ALT=843, NSQT=0, RALT=-0.5, ALTR=16, FQTY_1=3288, OIT_1=43.8212890625, OIT_2=42.47817993164063, AOA1=-2.4169760595703167, AOA2=-18.632681931152348, PTCH=0, FF_1=0, PSA=982.75, FF_2=0, FF_3=0, ROLL=0, FF_4=0, N1_1=1.75, N1_2=1.75, MACH=0, CAS=0, APFD=0, PH=1, CASM=300, TAS=0, VRTG=1.0038571357727049, LATG=-0.0246279239654541, PI=0.0, PS=29.03350830078125, N1_3=1.78125, EVNT=1, MRK=7, VIB_1=0.0, PT=982.71875, VHF1=1, VHF2=1, LGDN=0, LGUP=1, VIB_2=0.0, VHF3=1, PUSH=1, SHKR=1, MSQT_2=0, VIB_3=0.0, LONG=-0.008371949195861816, PLA_1=-1.0546857617187584, N1_4=1.46875, HYDY=0, HYDG=0, SMOK=0, CALT=0, VIB_4=0, PLA_2=-1.669904877929696, PLA_3=1.1425700187683103, PLA_4=-1.933576752929696, GMT_HOUR=45, GMT_MINUTE=165, GMT_SEC=90, ACMT=59, FQTY_2=0, OIT_3=42.47817993164063, OIT_4=43.8212890625, BLV=0, EAI=0, PAC

In [3]:
bucket_name = "iiaweb-s3-io-practice-bucket"

s3_resource = boto3.resource('s3', region_name='us-east-2')
bucket = s3_resource.Bucket(bucket_name)

Inspect bucket contents

In [13]:
contents = list(bucket.objects.all())
zipped_filekeys = [object.key for object in contents if '.gzip' in object.key]
print(len(zipped_filekeys))

47171

In [4]:
bucket_name = 'iiaweb-s3-io-practice-bucket'
key = '652/1/652200101092009.gzip'
print(bucket_name, key)

iiaweb-s3-io-practice-bucket 652/1/652200101092009.gzip

In [22]:
s3_client = boto3.client('s3')

# key = zipped_filekeys[0]
obj = s3_client.get_object(Bucket= bucket_name, Key= key)


print("Loading file: ", key)
pandas_df = pd.read_csv(obj['Body'], compression='gzip')
pandas_df.head()

Loading file:  652/1/652200101092009.gzip
             timestamp  FPAC  BLAC  CTAC  ...  EHRS_3  EHRS_2  ESN_1  ESN_2
0  2001-01-09 20:09:00     0     0     0  ...    6359    5633   7882   7833
1  2001-01-09 20:09:01     0     0     0  ...    6359    5633   7882   7833
2  2001-01-09 20:09:02     0     0     0  ...    6359    5633   7882   7833
3  2001-01-09 20:09:03     0     0     0  ...    6359    5633   7882   7833
4  2001-01-09 20:09:04     0     0     0  ...    6359    5633   7882   7833

[5 rows x 187 columns]

## Create Dictionary of Computation Times

In [42]:
times = {}

## Loading a Zipped CSV From S3 to Spark 

In [5]:
#Initialize a Master DF with the first file
started = datetime.datetime.now()

s3_client = boto3.client('s3')

# key = zipped_filekeys[0]

obj = s3_client.get_object(Bucket= bucket_name, Key= key)
df = spark.createDataFrame(pd.read_csv(obj['Body'], compression='gzip'))

finished = datetime.datetime.now()
elapsed = finished - started
print('Operation took: ', elapsed)

Operation took:  0:00:09.543860

In [43]:
times['s3_to_spark'] = elapsed

In [30]:
seconds = elapsed.seconds * len(zipped_filekeys)
minutes = seconds / 60

print(minutes)

3193.05

In [37]:
len(df)

An error was encountered:
object of type 'DataFrame' has no len()
Traceback (most recent call last):
TypeError: object of type 'DataFrame' has no len()



In [117]:
started = datetime.datetime.now()

#Initialize a Master DF with the first file
# obj = s3_client.get_object(Bucket= bucket_name, Key= key)
# df = spark.createDataFrame(pd.read_csv(obj['Body'], compression='gzip'))

#Concat subsequent files
for key in zipped_filekeys[11:100]:
    obj = s3_client.get_object(Bucket= bucket_name, Key= key)
    new = spark.createDataFrame(pd.read_csv(obj['Body'], compression='gzip'))
    df.union(new)

finished = datetime.datetime.now()
elapsed = finished - started
print('Operation took: ', elapsed)

Operation took:  0:23:43.075934

## List HDFS Files

In [6]:
hdfs.ls('/')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/DataValidation', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Experiments', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Jupyter', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/MattTest_Training_Datasets', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Models', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources']

In [29]:
hdfs.ls('/MattTest_Training_Datasets/')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/MattTest_Training_Datasets/README.md']

In [38]:
hdfs.ls('/Resources')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/.sparkStaging', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/README.md', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/spark-warehouse']

In [25]:
hdfs.ls('/Logs')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/Flink', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/PySpark', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/README.md', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/README_dump_test.md', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/Spark']

In [40]:
import stat
file_stat = hdfs.stat("Logs/README.md")
file_stat

StatResult(st_atime=1597975838, st_blksize=134217728, st_blocks=1, st_ctime=0, st_dev=0, st_gid='MattTest__Logs', st_ino=0, st_mode=488, st_mtime=1597344963, st_nlink=1, st_size=227, st_uid='MattTest__mmitchel', kind='file', name='hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/README.md', replication=3)

In [44]:
hdfs.stat('/Resources/NASA_Dashlink.parquet')

StatResult(st_atime=1598015761, st_blksize=134217728, st_blocks=1, st_ctime=0, st_dev=0, st_gid='MattTest__Resources', st_ino=0, st_mode=416, st_mtime=1598015761, st_nlink=1, st_size=4, st_uid='MattTest__mmitchel', kind='file', name='hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/NASA_Dashlink.parquet', replication=3)

In [146]:
hdfs.("/", 755)

In [49]:
hdfs.chmod("Resources/NASA_Dashlink.parquet", 777)

An error was encountered:
path hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/NASA_Dashlink.parquet not found
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/hdfs.py", line 518, in chmod
    hdfs_path = _expand_path(hdfs_path, project)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/hdfs.py", line 161, in _expand_path
    raise IOError("path %s not found" % hdfs_path)
OSError: path hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources/NASA_Dashlink.parquet not found



In [43]:
hdfs.dump("test", '/Resources/NASA_Dashlink.parquet')

In [6]:
started = datetime.datetime.now()

df.write.parquet('NASA_Dashlink_Sample1.parquet')

finished = datetime.datetime.now()
elapsed = finished - started

print('Operation took: ', elapsed)

An error was encountered:
An error occurred while calling o165.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=MattTest__mmitchel, access=WRITE, inode="user":hdfs:hadoop:drwxrwxr-t
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:520)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:337)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:227)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1788)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1773)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1755)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp$1.performTask(FSDirMkdirOp.java:103)
	at io.hops.transaction.handler.TransactionalRequestHandler.execute(TransactionalRequestHandler

In [142]:
hdfs.ls('/')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/DataValidation', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Experiments', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Jupyter', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/MattTest_Training_Datasets', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Models', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Resources']

In [143]:
hdfs.ls('Resources/')

An error was encountered:
[Errno 13] Permission denied
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/hdfs.py", line 735, in ls
    return hdfs.ls(hdfs_path, recursive=recursive)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/pydoop/hdfs/__init__.py", line 307, in ls
    dir_list = lsl(hdfs_path, user, recursive)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/pydoop/hdfs/__init__.py", line 288, in lsl
    dir_list = fs.list_directory(path_)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/pydoop/hdfs/fs.py", line 419, in list_directory
    return self.fs.list_directory(path)
PermissionError: [Errno 13] Permission denied



## Write to HDFS

In [144]:
started = datetime.datetime.now()


# filename = key
# df.write.save('key', format='parquet', mode='append')

# finished = datetime.datetime.now()
# elapsed = finished - started



logs_README = hdfs.load("Logs/README.md")
print("logs README: {}".format(logs_README.decode("utf-8")))
hdfs.dump("test", "Logs/README_dump_test.md")
logs_README_dumped = hdfs.load("Logs/README_dump_test.md")
print("logs README dumped: {}".format(logs_README_dumped.decode("utf-8")))



# times['SparkDf_to_HDFS'] = elapsed
print('Operation took: ', elapsed)

logs README: *This is an auto-generated README.md file for your Dataset!*
To replace it, go into your DataSet and edit the README.md file.

*Logs* DataSet
===

## Contains the logs for jobs that have been run through the Hopsworks platform.
logs README dumped: test
Operation took:  0:23:43.075934

In [145]:
hdfs.ls('Logs/')

['hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/Flink', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/PySpark', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/README.md', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/README_dump_test.md', 'hdfs://rpc.namenode.service.consul:8020/Projects/MattTest/Logs/Spark']

In [ ]:
hdfs.dump(df, "")

## Load From Parquet

## Hive

In [31]:
from hops import hdfs as hopsfs
PROJECT_NAME = hopsfs.project_name()

In [32]:
PROJECT_NAME

'MattTest'

## Show Tables

In [33]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

## Create Initial Table

In [34]:
df.write.format("ORC").mode("overwrite").saveAsTable("NASA_Dashlink")

An error was encountered:
An error occurred while calling o221.saveAsTable.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=MattTest__mmitchel, access=WRITE, inode="warehouse":hive:hadoop:drwxr-xr-t
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:520)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:337)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:227)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1788)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1773)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1755)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp$1.performTask(FSDirMkdirOp.java:103)
	at io.hops.transaction.handler.TransactionalRequestHandler.execute(TransactionalReque

# Times

In [45]:
times

{'s3_to_spark': datetime.timedelta(0, 9, 401443)}

## Sample

In [56]:
df.show()

+-------------------+----+----+----+---+---+-----+-----+-----+-----+---+---+---+----+---+----+---+---+----+---+---+----+------+----+------+-----------------+-----------------+-------------------+-------------------+----+----+---------+----+----+----+----+----+----+----+---+----+---+----+---+------------------+--------------------+-------+------------------+-------+----+---+-----+---------+----+----+----+----+-----+----+----+----+------+-----+--------------------+-------------------+----+----+----+----+----+-----+-------------------+------------------+------------------+--------+----------+-------+----+------+-----------------+-----------------+---+---+----+-------------------+----+----+----+----+----+----+---+----+----+----+----+-----+-----+---------+---+-------------------+------+-----+-----+------+-----------------+-----------------+----+----+----+----+----+---+------+-----+----+---+----+----+----+----+----+-----+-----+--------------------+--------------------+---------+-------------

## Initial Clustering

In [100]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [60]:
df.schema.names
# df.printSchema()

['timestamp', 'FPAC', 'BLAC', 'CTAC', 'TH', 'MH', 'EGT_1', 'EGT_2', 'EGT_3', 'EGT_4', 'IVV', 'GS', 'TRK', 'TRKM', 'DA', 'POVT', 'WS', 'MW', 'DFGS', 'WD', 'ALT', 'NSQT', 'RALT', 'ALTR', 'FQTY_1', 'OIT_1', 'OIT_2', 'AOA1', 'AOA2', 'PTCH', 'FF_1', 'PSA', 'FF_2', 'FF_3', 'ROLL', 'FF_4', 'N1_1', 'N1_2', 'MACH', 'CAS', 'APFD', 'PH', 'CASM', 'TAS', 'VRTG', 'LATG', 'PI', 'PS', 'N1_3', 'EVNT', 'MRK', 'VIB_1', 'PT', 'VHF1', 'VHF2', 'LGDN', 'LGUP', 'VIB_2', 'VHF3', 'PUSH', 'SHKR', 'MSQT_2', 'VIB_3', 'LONG', 'PLA_1', 'N1_4', 'HYDY', 'HYDG', 'SMOK', 'CALT', 'VIB_4', 'PLA_2', 'PLA_3', 'PLA_4', 'GMT_HOUR', 'GMT_MINUTE', 'GMT_SEC', 'ACMT', 'FQTY_2', 'OIT_3', 'OIT_4', 'BLV', 'EAI', 'PACK', 'AOAI', 'AOAC', 'BAL1', 'APUF', 'TOCW', 'BAL2', 'WSHR', 'WOW', 'N2_1', 'N2_2', 'N2_3', 'N2_4', 'TAT', 'SAT', 'N1T', 'N1C', 'RUDD', 'FQTY_3', 'OIP_1', 'OIP_2', 'FQTY_4', 'CRSS', 'HDGS', 'ALTS', 'SNAP', 'CASS', 'N1CO', 'VSPS', 'MNS', 'MSQT_1', 'VMODE', 'LMOD', 'A_T', 'CCPC', 'CCPF', 'RUDP', 'CWPC', 'CWPF', 'OIP_3', 'OI

In [61]:
df.schema.names[1:]

['FPAC', 'BLAC', 'CTAC', 'TH', 'MH', 'EGT_1', 'EGT_2', 'EGT_3', 'EGT_4', 'IVV', 'GS', 'TRK', 'TRKM', 'DA', 'POVT', 'WS', 'MW', 'DFGS', 'WD', 'ALT', 'NSQT', 'RALT', 'ALTR', 'FQTY_1', 'OIT_1', 'OIT_2', 'AOA1', 'AOA2', 'PTCH', 'FF_1', 'PSA', 'FF_2', 'FF_3', 'ROLL', 'FF_4', 'N1_1', 'N1_2', 'MACH', 'CAS', 'APFD', 'PH', 'CASM', 'TAS', 'VRTG', 'LATG', 'PI', 'PS', 'N1_3', 'EVNT', 'MRK', 'VIB_1', 'PT', 'VHF1', 'VHF2', 'LGDN', 'LGUP', 'VIB_2', 'VHF3', 'PUSH', 'SHKR', 'MSQT_2', 'VIB_3', 'LONG', 'PLA_1', 'N1_4', 'HYDY', 'HYDG', 'SMOK', 'CALT', 'VIB_4', 'PLA_2', 'PLA_3', 'PLA_4', 'GMT_HOUR', 'GMT_MINUTE', 'GMT_SEC', 'ACMT', 'FQTY_2', 'OIT_3', 'OIT_4', 'BLV', 'EAI', 'PACK', 'AOAI', 'AOAC', 'BAL1', 'APUF', 'TOCW', 'BAL2', 'WSHR', 'WOW', 'N2_1', 'N2_2', 'N2_3', 'N2_4', 'TAT', 'SAT', 'N1T', 'N1C', 'RUDD', 'FQTY_3', 'OIP_1', 'OIP_2', 'FQTY_4', 'CRSS', 'HDGS', 'ALTS', 'SNAP', 'CASS', 'N1CO', 'VSPS', 'MNS', 'MSQT_1', 'VMODE', 'LMOD', 'A_T', 'CCPC', 'CCPF', 'RUDP', 'CWPC', 'CWPF', 'OIP_3', 'OIP_4', 'LOC', 

In [67]:
df.take(1)

[Row(timestamp='2001-01-09 20:46:00', FPAC=0, BLAC=0, CTAC=0, TH=0, MH=0, EGT_1=96.0, EGT_2=109.0, EGT_3=129.0, EGT_4=139.0, IVV=0, GS=0, TRK=0, TRKM=0, DA=0, POVT=0, WS=0, MW=0, DFGS=1, WD=0, ALT=851, NSQT=0, RALT=-0.375, ALTR=16, FQTY_1=3000, OIT_1=53.22299194335938, OIT_2=55.9091796875, AOA1=-2.4169760595703167, AOA2=-18.632681931152348, PTCH=0, FF_1=0, PSA=982.4375, FF_2=0, FF_3=0, ROLL=0, FF_4=0, N1_1=1.75, N1_2=1.75, MACH=0, CAS=0, APFD=0, PH=2, CASM=300, TAS=0, VRTG=1.0038571357727049, LATG=-0.02513599395751953, PI=0.0, PS=29.025691986083984, N1_3=1.78125, EVNT=1, MRK=7, VIB_1=0, PT=982.46875, VHF1=1, VHF2=1, LGDN=0, LGUP=1, VIB_2=0, VHF3=1, PUSH=1, SHKR=1, MSQT_2=0, VIB_3=0, LONG=-1.0832999944686892, PLA_1=-0.7031232617187584, N1_4=1.5, HYDY=0, HYDG=0, SMOK=0, CALT=0, VIB_4=0.0, PLA_2=-0.7910138867187584, PLA_3=1.3183499574661257, PLA_4=-1.933576752929696, GMT_HOUR=45, GMT_MINUTE=165, GMT_SEC=90, ACMT=59, FQTY_2=0, OIT_3=49.19369506835938, OIT_4=46.50747680664063, BLV=0, EAI=0,

In [75]:
df["FPAC"].show()

An error was encountered:
'Column' object is not callable
Traceback (most recent call last):
TypeError: 'Column' object is not callable



In [112]:
started = datetime.datetime.now()

assembler = VectorAssembler(inputCols=df.schema.names[1:], outputCol="X_vec")

scaler = MinMaxScaler(inputCol="X_vec", outputCol="features")


pipeline = Pipeline(stages=[assembler, scaler])
scalerModel = pipeline.fit(df)
scaledData = scalerModel.transform(df)


finished = datetime.datetime.now()
elapsed = finished - started
print('Process took: {}'.format(elapsed))

# Compute summary statistics and generate MinMaxScalerModel
# scalerModel = scaler.fit(df)
# df = scalerModel.transform(df)

Process took: 0:00:09.202690

In [113]:
scaledData.take(1)

[Row(timestamp='2001-01-09 20:46:00', FPAC=0, BLAC=0, CTAC=0, TH=0, MH=0, EGT_1=96.0, EGT_2=109.0, EGT_3=129.0, EGT_4=139.0, IVV=0, GS=0, TRK=0, TRKM=0, DA=0, POVT=0, WS=0, MW=0, DFGS=1, WD=0, ALT=851, NSQT=0, RALT=-0.375, ALTR=16, FQTY_1=3000, OIT_1=53.22299194335938, OIT_2=55.9091796875, AOA1=-2.4169760595703167, AOA2=-18.632681931152348, PTCH=0, FF_1=0, PSA=982.4375, FF_2=0, FF_3=0, ROLL=0, FF_4=0, N1_1=1.75, N1_2=1.75, MACH=0, CAS=0, APFD=0, PH=2, CASM=300, TAS=0, VRTG=1.0038571357727049, LATG=-0.02513599395751953, PI=0.0, PS=29.025691986083984, N1_3=1.78125, EVNT=1, MRK=7, VIB_1=0, PT=982.46875, VHF1=1, VHF2=1, LGDN=0, LGUP=1, VIB_2=0, VHF3=1, PUSH=1, SHKR=1, MSQT_2=0, VIB_3=0, LONG=-1.0832999944686892, PLA_1=-0.7031232617187584, N1_4=1.5, HYDY=0, HYDG=0, SMOK=0, CALT=0, VIB_4=0.0, PLA_2=-0.7910138867187584, PLA_3=1.3183499574661257, PLA_4=-1.933576752929696, GMT_HOUR=45, GMT_MINUTE=165, GMT_SEC=90, ACMT=59, FQTY_2=0, OIT_3=49.19369506835938, OIT_4=46.50747680664063, BLV=0, EAI=0,

In [109]:
scaledData.select('X_Transformed').take(1)

[Row(X_Transformed=DenseVector([0.5, 0.5, 0.5, 0.5, 0.5, 0.9667, 1.0, 0.8857, 0.0043, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.25, 0.5, 0.5, 0.6154, 1.0, 1.0, 0.997, 0.5, 0.25, 0.5, 0.5, 0.7273, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9995, 0.999, 0.0, 0.8999, 0.5, 1.0, 1.0, 0.5, 0.75, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6667, 1.0, 0.0, 0.5, 0.5, 0.5, 0.2, 0.5, 0.997, 0.9003, 0.0, 0.0, 0.0, 0.5, 0.5, 0.2, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 1.0, 0.5, 0.9975, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9339, 0.9421, 0.9995, 0.9995, 0.9995, 0.5, 0.0, 0.9694, 0.7214, 0.5, 0.0, 0.9265, 0.9205, 0.9963, 0.9995, 0.966, 0.32, 1.0, 0.9995, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.8333, 0.9965, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.8436, 0

## KMeans Initialization

In [116]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

started = datetime.datetime.now()

# Trains a k-means model.
kmeans = KMeans().setK(20).setSeed(1)
model = kmeans.fit(scaledData)

# Make predictions
predictions = model.transform(scaledData)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))


# Evaluate clustering.
cost = model.computeCost(scaledData)
print("Within Set Sum of Squared Errors = " + str(cost))

# Shows the result.
print("Cluster Centers: ")
ctr=[]
centers = model.clusterCenters()
# for center in centers:
#     ctr.append(center)
#     print(center)

finished = datetime.datetime.now()
elapsed = finished - started
print('Process took: {}'.format(elapsed))

Silhouette with squared euclidean distance = 0.3954954345939706
Within Set Sum of Squared Errors = 1372.8948604032742
Cluster Centers: 
Process took: 0:00:17.127464

## Misc Investigations

In [76]:
df.createOrReplaceTempView("df")
spark.sql("select count(*) from df").collect()

[Row(count(1)=1895)]

In [78]:
df.select('FPAC').dtypes

[('FPAC', 'bigint')]

In [94]:
from pyspark.sql.types import IntegerType, DoubleType, ShortType, ByteType

In [95]:
df = df.withColumn("FPAC",df["FPAC"].cast(ByteType()))
df.select('FPAC').dtypes

[('FPAC', 'tinyint')]